In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7018222609485249541
xla_global_id: -1
]


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Conv2D, MaxPooling2D, Dropout, Add, BatchNormalization, Activation, Reshape
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, mean_squared_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

my_seed = 1027

tf.random.set_seed(my_seed)
np.random.seed(my_seed)

import os
import ast
import csv

from tqdm import tqdm
def tqdm_close():
    for instance in tqdm._instances:
        instance.close()
import multiprocessing as mp
print('cpu 개수는:', mp.cpu_count())
from joblib import Parallel, delayed        

cpu 개수는: 12


In [43]:
# input layer
input_shape_1 = (4, 10000, 1) # resampling raw data
input_shape_2 = (100, 100, 4) # frequency featuring

input_layer_1 = Input(shape=input_shape_1)
input_layer_2 = Input(shape=input_shape_2)

# 1D CONV # raw feature extract
# filter  100개로 -> log mel spectrogram + mfcc의 구성 요소 수와 동일 : 각 필터가 주파수 특성과 연관
x1 = Conv2D(100, (1, 1), activation='relu')(input_layer_1) # (4ch, 10000t, 100feature)
x1 = Conv2D(100, (1, 1), activation='relu')(x1) # (4, 10000, 100)
x1 = MaxPooling2D((1, 100))(x1) # (4, 100t, 100f)
x1 = tf.transpose(x1, perm=[0, 3, 2, 1]) # (100f, 100t, 4)

# MLnet
x1 = Conv2D(32, (4, 4), (2, 2), activation='relu')(x1) # (49, 49, 32)
x1 = Conv2D(32, (4, 4), (2, 2), activation='relu')(x1) # (23, 23, 32)
x1 = MaxPooling2D((2, 2), (2, 2))(x1) #(11, 11, 32)
x1 = Flatten()(x1) #(3872)

# MLnet
x2 = Conv2D(32, (4, 4), (2, 2), activation='relu')(input_layer_2) # (49, 49, 32)
x2 = Conv2D(32, (4, 4), (2, 2), activation='relu')(x2) # (23, 23, 32)
x2 = MaxPooling2D((2, 2), (2, 2))(x2) #(11, 11, 32)
x2 = Flatten()(x2) #(3872)

x3 = Add()([x1, x2])
x3 = Dense(512, activation='relu')(x3)
x3 = Dense(64, activation='relu')(x3)
output_layer = Dense(3)(x3)

model = Model(inputs=[input_layer_1, input_layer_2], outputs=output_layer)


initial_learning_rate, min_learning_rate  = 0.001, 0.000000001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,  # 초기 학습률
    decay_steps=10000,      # 학습률을 감소시킬 스텝 수
    decay_rate=0.9,         # 학습률을 줄일 비율
    staircase=False
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

losses = ['mean_squared_error', 'mean_squared_error', tf.losses.BinaryCrossentropy(), ]
loss_weights =  [1, 1, 0.1]

model.compile(optimizer=optimizer, loss=losses, loss_weights=loss_weights)

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_43 (InputLayer)          [(None, 4, 10000, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_76 (Conv2D)             (None, 4, 10000, 10  200         ['input_43[0][0]']               
                                0)                                                                
                                                                                                  
 conv2d_77 (Conv2D)             (None, 4, 10000, 10  10100       ['conv2d_76[0][0]']              
                                0)                                                          

In [2]:
df1 = pd.read_csv('siren_class.csv')
df0 = pd.read_csv('other_class.csv')
df = pd.concat([df0,df1], axis=0)

In [51]:
print(df.shape)
print(df.columns)

(5285, 25)
Index(['ch1_raw', 'ch2_raw', 'ch3_raw', 'ch4_raw', 'ch1_rsp', 'ch2_rsp',
       'ch3_rsp', 'ch4_rsp', 'ch1_rt', 'ch2_rt', 'ch3_rt', 'ch4_rt', 'mean',
       'x', 'y', 'i', 'deg', 'sin', 'cos', 'dt', 'cls', 'ch1_fre', 'ch2_fre',
       'ch3_fre', 'ch4_fre'],
      dtype='object')


In [12]:
df[['ch1_rsp', 'ch2_rsp', 'ch3_rsp', 'ch4_rsp']]

,ch1_rsp,ch2_rsp,ch3_rsp,ch4_rsp
0,"[131.768, 137.635, 142.722, 147.032, 150.589, ...","[122.653, 128.084, 132.783, 136.754, 140.018, ...","[173.051, 180.608, 187.113, 192.571, 197.012, ...","[114.327, 119.51, 124.032, 127.896, 131.121, 1..."
1,"[140.894, 146.984, 152.207, 156.567, 160.09, 1...","[165.034, 172.254, 178.475, 183.699, 187.958, ...","[123.228, 128.491, 132.985, 136.713, 139.698, ...","[165.757, 172.871, 178.957, 184.018, 188.084, ..."
2,"[248.296, 259.012, 268.198, 275.861, 282.044, ...","[112.943, 117.819, 122.001, 125.49, 128.308, 1...","[217.005, 226.557, 234.805, 241.754, 247.442, ...","[176.165, 183.87, 190.506, 196.079, 200.619, 2..."
3,"[136.828, 142.84, 148.028, 152.395, 155.965, 1...","[134.333, 140.267, 145.397, 149.726, 153.277, ...","[96.636, 100.843, 104.461, 107.492, 109.953, 1...","[109.245, 114.039, 118.173, 121.65, 124.489, 1..."
4,"[76.746, 80.231, 83.274, 85.877, 88.055, 89.83...","[84.06, 88.015, 91.51, 94.547, 97.14, 99.315, ...","[59.748, 62.568, 65.062, 67.233, 69.091, 70.65...","[50.851, 53.239, 55.347, 57.178, 58.741, 60.05..."
...,...,...,...,...
3096,"[40.284, 42.062, 43.6, 44.897, 45.962, 46.806,...","[45.057, 47.004, 48.673, 50.067, 51.192, 52.06...","[47.841, 49.984, 51.846, 53.428, 54.739, 55.79...","[49.859, 52.036, 53.911, 55.484, 56.765, 57.77..."
3097,"[70.117, 73.167, 75.788, 77.983, 79.764, 81.15...","[77.56, 80.988, 83.953, 86.455, 88.51, 90.14, ...","[64.25, 67.073, 69.509, 71.559, 73.235, 74.558...","[88.923, 92.704, 95.927, 98.594, 100.723, 102...."
3098,"[49.944, 52.108, 53.965, 55.518, 56.774, 57.74...","[57.156, 59.757, 62.028, 63.973, 65.6, 66.927,...","[80.329, 83.896, 86.985, 89.599, 91.752, 93.46...","[65.295, 68.313, 70.963, 73.248, 75.177, 76.77..."
3099,"[61.068, 63.776, 66.119, 68.101, 69.73, 71.028...","[42.233, 44.06, 45.628, 46.938, 47.997, 48.821...","[63.817, 66.674, 69.156, 71.264, 73.01, 74.414...","[65.567, 68.588, 71.238, 73.52, 75.443, 77.029..."


In [18]:
def np_ast(str_list):
    return np.array(ast.literal_eval(str_list))

df_input1 = df[['ch1_rsp', 'ch2_rsp', 'ch3_rsp', 'ch4_rsp']].values
len_df = len(df_input1)

train_input1_list = []
for i in tqdm(range(len_df)):
    train_input1_sub_list = []
    for col in range(4): 
        train_input1_sub_list.append(ast.literal_eval(df_input1[i][col]))
    train_input1_list.append(train_input1_sub_list)   

train_input1_array = np.array(train_input1_list).reshape((5285, 4, 10000, 1))
tqdm_close()

100%|██████████████████████████████████████████████████████████████████████████████| 5285/5285 [32:51<00:00,  2.68it/s]


ValueError: cannot reshape array of size 211400000 into shape (3,4,10000,1)

In [ ]:
df_input2 = df[['ch1_fre', 'ch2_fre', 'ch3_fre', 'ch4_fre']]
a = df_input2.apply(lambda row: [ast.literal_eval(value) for value in row], axis=1)

In [61]:
df_input2 = df[['ch1_fre', 'ch2_fre', 'ch3_fre', 'ch4_fre']].values
len_df = len(df_input2)

train_input2_list = []
for i in tqdm(range(len_df)):
    train_input2_sub_list = []
    for col in range(4): 
        train_input2_sub_list.append(np.array(ast.literal_eval(df_input2[i][col])))
    train_input2_list.append(np.array(train_input2_sub_list))

train_input2_array = np.array(train_input2_list)
tqdm_close()

  0%|▎                                                                             | 25/5285 [00:28<1:40:06,  1.14s/it]


KeyboardInterrupt: 

In [27]:
output_array = np.array(df[['x', 'y', 'cls']].values)

In [ ]:
# 데이터를 준비하고 분할
x_train_1 = input_data_array_t
x_train_2 = input_feature_array
y_train = input_label_array  # 타겟 데이터

x_train_1, x_val_1, x_train_2, x_val_2, y_train, y_val = train_test_split(x_train_1, x_train_2, y_train, test_size=0.2, random_state=my_seed)

# 모델 학습
batch_size = 128
epochs = 10

# 입력 데이터 목록을 사용하여 모델 학습
model.fit([x_train_1, x_train_2], y_train, validation_data=([x_val_1, x_val_2], y_val), batch_size=batch_size, epochs=epochs)